# 1.Install Dependencies

In [1]:
!pip install tensorflow mediapipe matplotlib scikit-learn opencv-python

# 2. import Dependencies

In [2]:
import cv2
import mediapipe as mp
import numpy as np 
import os 
import matplotlib.pyplot as plt
import time
from tensorflow.keras.models import load_model,save_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# Holistic model from mediapipe
mp_holistic=mp.solutions.holistic
# Drawing utilities from mediapipe
mp_drawing=mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image,model):
    #Convert color from bgr to rgb
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # Image is no longer writeable
    image.flags.writeable=False
    # Process image by predict keypoints
    results=model.process(image)
    # Image is now writeable 
    image.flags.writeable=True
    #Convert color from rgb to bgr
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections and landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections and landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections and landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections and landmarks

In [6]:
#same as draw_landmarks but with diffrent style
def draw_styled_landmarks(image, results):
    # Draw face connections and landmarks
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections and landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections and landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections and landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
# lets test what we have done so far if you run this cell the camera will open automatically and to close it just press q 
# if the camera didn't open try to change the value 0 in the line below to 1 or 2
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read frame
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('My camera', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [4]:
# if the camera didn't close try to restart the kernel and run this cell to close it 
cap.release()
cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [ ]:
#those lines of codes make us stock keypoints for each part 
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() 
                   #33*4 because they are 33 landmarks for pose and for each one we have x,y,z,visibility
                   if results.pose_landmarks else np.zeros(33*4)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
                   #468*3 because they are 33 landmarks for face and for each one we have x,y,z
                   if results.face_landmarks else np.zeros(468*3)
leftHand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() 
                   #21*3 because they are 33 landmarks for hand and for each one we have x,y,z
                   if results.left_hand_landmarks else np.zeros(21*3)
rightHand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [7]:
#we wrap up all this in a function so we can use it later to stock keypoints when we collect data 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    leftHand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rightHand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, leftHand, rightHand])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
len(result_test)

In [ ]:
468*3+33*4+21*3+21*3

In [ ]:
#this is to save a numpy array to computer because later we will store our keypoints data in numpy arrays
np.save('0', result_test)

In [9]:
#this is to load a numpy array from computer
np.load('0.npy')

array([ 0.49521989,  0.57799989, -1.02819157, ...,  0.        ,
        0.        ,  0.        ])

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH=os.path.join('MP_Data')
# Actions that we try to detect u can change it 
actions=np.array(['hello','thanks','iloveyou'])
# Thirty videos worth of data
no_sequences=30
# Videos are going to be 30 frames in length
sequence_length=30

In [ ]:
#creation 30 folders to store 30 images for each action
for action in actions:
    for sequences in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequences)))
        except :
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
#lets collect some data 
#if u run this cell the camera will open again but this time to collect data for our actions
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.putText??

# 6. Preprocess Data and Create Labels and Features

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

In [10]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [11]:
#load our data back 
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(90, 30, 1662)

In [13]:
X = np.array(sequences)

In [14]:
np.array(labels).shape

(90,)

In [15]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [19]:
y_test.shape

(5, 3)

# 7. Build and Train LSTM Neural Network

In [20]:
#using tensorboard to visualize training plots 
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
#callback to save the best model during training which has the maximum categorical accuracy
checkpoint_callback = ModelCheckpoint(
    './best_weights',
    monitor='categorical_accuracy',
    mode = 'max',
    verbose=1,
    save_best_only=True,
    )

In [22]:
#build the model using the sequential api 
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [23]:
#compile the model 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [69]:
#train the model
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,checkpoint_callback])

Epoch 1/2000
3/3 [==============================] - ETA: 0s - loss: 2.0247 - categorical_accuracy: 0.2941
Epoch 1: categorical_accuracy improved from -inf to 0.29412, saving model to .\best_weights
INFO:tensorflow:Assets written to: .\best_weights\assets


INFO:tensorflow:Assets written to: .\best_weights\assets


3/3 [==============================] - 11s 3s/step - loss: 2.0247 - categorical_accuracy: 0.2941
Epoch 2/2000
3/3 [==============================] - ETA: 0s - loss: 8.8095 - categorical_accuracy: 0.2353
Epoch 2: categorical_accuracy did not improve from 0.29412
3/3 [==============================] - 0s 34ms/step - loss: 8.8095 - categorical_accuracy: 0.2353
Epoch 3/2000
3/3 [==============================] - ETA: 0s - loss: 25.9143 - categorical_accuracy: 0.3647
Epoch 3: categorical_accuracy improved from 0.29412 to 0.36471, saving model to .\best_weights
INFO:tensorflow:Assets written to: .\best_weights\assets


INFO:tensorflow:Assets written to: .\best_weights\assets


3/3 [==============================] - 7s 3s/step - loss: 25.9143 - categorical_accuracy: 0.3647
Epoch 4/2000
3/3 [==============================] - ETA: 0s - loss: 8.3517 - categorical_accuracy: 0.2824
Epoch 4: categorical_accuracy did not improve from 0.36471
3/3 [==============================] - 0s 39ms/step - loss: 8.3517 - categorical_accuracy: 0.2824
Epoch 5/2000
3/3 [==============================] - ETA: 0s - loss: 12.6283 - categorical_accuracy: 0.2471
Epoch 5: categorical_accuracy did not improve from 0.36471
3/3 [==============================] - 0s 40ms/step - loss: 12.6283 - categorical_accuracy: 0.2471
Epoch 6/2000
2/3 [===================>..........] - ETA: 0s - loss: 10.8050 - categorical_accuracy: 0.3438
Epoch 6: categorical_accuracy did not improve from 0.36471
3/3 [==============================] - 0s 50ms/step - loss: 11.9142 - categorical_accuracy: 0.3294
Epoch 7/2000
3/3 [==============================] - ETA: 0s - loss: 9.9705 - categorical_accuracy: 0.2471 
Epo

...

Epoch 1796/2000
3/3 [==============================] - ETA: 0s - loss: 3.4153e-05 - categorical_accuracy: 1.0000
Epoch 1796: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 31ms/step - loss: 3.4153e-05 - categorical_accuracy: 1.0000
Epoch 1797/2000
3/3 [==============================] - ETA: 0s - loss: 3.4098e-05 - categorical_accuracy: 1.0000
Epoch 1797: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 32ms/step - loss: 3.4098e-05 - categorical_accuracy: 1.0000
Epoch 1798/2000
3/3 [==============================] - ETA: 0s - loss: 3.4054e-05 - categorical_accuracy: 1.0000
Epoch 1798: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 28ms/step - loss: 3.4054e-05 - categorical_accuracy: 1.0000
Epoch 1799/2000
3/3 [==============================] - ETA: 0s - loss: 3.4018e-05 - categorical_accuracy: 1.0000
Epoch 1799: categorical_accuracy did not improve from 1.

3/3 [==============================] - 0s 25ms/step - loss: 3.1805e-05 - categorical_accuracy: 1.0000
Epoch 1826/2000
3/3 [==============================] - ETA: 0s - loss: 3.1421e-05 - categorical_accuracy: 1.0000
Epoch 1826: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 28ms/step - loss: 3.1421e-05 - categorical_accuracy: 1.0000
Epoch 1827/2000
3/3 [==============================] - ETA: 0s - loss: 3.1338e-05 - categorical_accuracy: 1.0000
Epoch 1827: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 30ms/step - loss: 3.1338e-05 - categorical_accuracy: 1.0000
Epoch 1828/2000
3/3 [==============================] - ETA: 0s - loss: 3.1251e-05 - categorical_accuracy: 1.0000
Epoch 1828: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 34ms/step - loss: 3.1251e-05 - categorical_accuracy: 1.0000
Epoch 1829/2000
3/3 [==============================] - ETA: 0s - lo

Epoch 1885/2000
3/3 [==============================] - ETA: 0s - loss: 2.6094e-05 - categorical_accuracy: 1.0000
Epoch 1885: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 30ms/step - loss: 2.6094e-05 - categorical_accuracy: 1.0000
Epoch 1886/2000
3/3 [==============================] - ETA: 0s - loss: 2.5941e-05 - categorical_accuracy: 1.0000
Epoch 1886: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 31ms/step - loss: 2.5941e-05 - categorical_accuracy: 1.0000
Epoch 1887/2000
3/3 [==============================] - ETA: 0s - loss: 2.5947e-05 - categorical_accuracy: 1.0000
Epoch 1887: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 31ms/step - loss: 2.5947e-05 - categorical_accuracy: 1.0000
Epoch 1888/2000
3/3 [==============================] - ETA: 0s - loss: 2.5801e-05 - categorical_accuracy: 1.0000
Epoch 1888: categorical_accuracy did not improve from 1.

3/3 [==============================] - 0s 33ms/step - loss: 2.4149e-05 - categorical_accuracy: 1.0000
Epoch 1915/2000
3/3 [==============================] - ETA: 0s - loss: 2.4024e-05 - categorical_accuracy: 1.0000
Epoch 1915: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 32ms/step - loss: 2.4024e-05 - categorical_accuracy: 1.0000
Epoch 1916/2000
3/3 [==============================] - ETA: 0s - loss: 2.3962e-05 - categorical_accuracy: 1.0000
Epoch 1916: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 36ms/step - loss: 2.3962e-05 - categorical_accuracy: 1.0000
Epoch 1917/2000
3/3 [==============================] - ETA: 0s - loss: 2.3909e-05 - categorical_accuracy: 1.0000
Epoch 1917: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 36ms/step - loss: 2.3909e-05 - categorical_accuracy: 1.0000
Epoch 1918/2000
3/3 [==============================] - ETA: 0s - lo

Epoch 1974/2000
3/3 [==============================] - ETA: 0s - loss: 2.0809e-05 - categorical_accuracy: 1.0000
Epoch 1974: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 34ms/step - loss: 2.0809e-05 - categorical_accuracy: 1.0000
Epoch 1975/2000
3/3 [==============================] - ETA: 0s - loss: 2.0744e-05 - categorical_accuracy: 1.0000
Epoch 1975: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 31ms/step - loss: 2.0744e-05 - categorical_accuracy: 1.0000
Epoch 1976/2000
3/3 [==============================] - ETA: 0s - loss: 2.0761e-05 - categorical_accuracy: 1.0000
Epoch 1976: categorical_accuracy did not improve from 1.00000
3/3 [==============================] - 0s 34ms/step - loss: 2.0761e-05 - categorical_accuracy: 1.0000
Epoch 1977/2000
3/3 [==============================] - ETA: 0s - loss: 2.0622e-05 - categorical_accuracy: 1.0000
Epoch 1977: categorical_accuracy did not improve from 1.

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 596675 

# 8. Load the best Model

In [25]:
#load the best model
best_model=load_model('./best_weights')

In [ ]:
#u can also save weights of the model 
#but note if u use this u are saving the last version of the model during training not the best one 
model.save('action.h5')

In [ ]:
#load the weights
model.load_weights('action.h5')

In [26]:
#evaluate it on the test set
best_model.evaluate(X_test,y_test)


1/1 [==============================] - 2s 2s/step - loss: 0.0284 - categorical_accuracy: 1.0000


[0.0283980630338192, 1.0]

# 9. Make Predictions

In [27]:
res = best_model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [28]:
actions[np.argmax(res[3])]

'iloveyou'

In [30]:
actions[np.argmax(y_test[3])]

'iloveyou'

# 10. Evaluation using Confusion Matrix and Accuracy

In [31]:
yhat = best_model.predict(X_test)

1/1 [==============================] - 0s 43ms/step


In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [34]:
accuracy_score(ytrue, yhat)

1.0

# 11. Test in Real Time

In [35]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = best_model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()